# User Testing Planning/Coding Notebook

### Objectives:
- Plan user testing
- Code scripts to generate test maps:
    - Randomly pull maps/music from 30 songs with maps for all difficulty levels from Beat Saver (tried all >70% rating but not enough were rated)
    - Strip obstacles/lighting events from human made maps and set background environment all the same
    - Map all songs with each model type
    - Automatically categorize songs/beat structure?
- For each user/volunteer:
    - Randomly assign Random, HMM, Segmented HMM, Rate-Seg HMM, or Human-made
    - Record email address, user-ID, and assigned map information (song, model/human, difficulty) in pd DataFrame
    - Send assignment to user    

In [27]:
import pandas as pd
import numpy as np
import json
import requests
import zipfile
from io import BytesIO
import pickle
import src.beatmapsynth

In [2]:
with open("../../data/metadata.pkl", 'rb') as file:
    metadata = pickle.load(file)

In [3]:
potential_songs = []

for x in metadata:
    if all(value == True for value in x['metadata']['difficulties'].values()):
        potential_songs.append(x)

In [4]:
len(potential_songs)

556

In [5]:
test_songs = np.random.choice(potential_songs, 30, replace = False)

For each test song in the list:
1. Download with key and unzip
2. For each difficulty level:
    - Map with each model
    - Edit/strip human map
    - Save zip folder

In [6]:
keys = []
for x in test_songs:
    keys.append(x['key'])

In [7]:
def open_map_file(difficulty):
    """This function opens the map file listed in the info.dat file for the specificed difficulty level."""
    with open('./temp/info.dat', 'rb') as i:
        info = json.load(i)
    for x in info['_difficultyBeatmapSets']:
        if x['_beatmapCharacteristicName'] == 'Standard':
            for y in x['_difficultyBeatmaps']:
                if y['_difficulty'] == difficulty:
                    file_name = y['_beatmapFilename']
                    with open(f"./temp/{file_name}", 'rb') as f:
                        map_file = json.load(f)
                
    return map_file

In [8]:
def download_song_and_map(key):
    """Downloads the zipped folder of song and mapping data from the beatsaber api. Extracts files to a 'temp' folder 
    in the local directory."""
    response = requests.get(f"https://beatsaver.com/api/download/key/{key}")
    if response.status_code == 200:
        content_as_file = BytesIO(response.content)
        zip_file = ZipFile(content_as_file)
        for x in zip_file.filelist:
            print(zip_file.extract(x.filename, path = 'temp'))
        return response.status_code
    else:
        return print(f"API call failed at {key} with error code {response.status_code}")

In [14]:
download_song_and_map(keys[0])

API call failed at 501f with error code 403


Requests isn't working. Used Bash in terminal and wget to download zip folders instead.

In [17]:
with open("../../data/raw/keys.txt", 'w') as w:
    w.write(str(keys))

Needed a few extra keys, three keys didn't work anymore.

In [21]:
extra = np.random.choice(potential_songs, 3, replace = False)

In [22]:
extra_keys = []
for x in extra:
    extra_keys.append(x['key'])

In [23]:
extra_keys

['1ff6', '4f00', '2f1d']

In [24]:
for x in extra_keys:
    keys.append(x)

In [32]:
with open("../../data/raw/501f", 'rb') as zipfolder:
    zip_file = zipfile.ZipFile(zipfolder)
    for x in zip_file.filelist:
        print(zip_file.extract(x.filename, path = 'temp'))

temp/cover.jpg
temp/ExpertPlus.dat
temp/Expert.dat
temp/Hard.dat
temp/Normal.dat
temp/Easy.dat
temp/song.egg
temp/info.dat


In [34]:
easy = open_map_file('Easy')

In [36]:
easy.keys()

dict_keys(['_version', '_BPMChanges', '_events', '_notes', '_obstacles', '_bookmarks'])

In [37]:
easy['_events'] = []

In [38]:
easy['_obstacles'] = []

In [40]:
easy['_bookmarks'] = []

In [43]:
with open('./temp/info.dat', 'rb') as i:
    info = json.load(i)

In [44]:
info

{'_version': '2.0.0',
 '_songName': 'Calling',
 '_songSubName': '',
 '_songAuthorName': 'Zekk',
 '_levelAuthorName': 'Hyunchen',
 '_beatsPerMinute': 180,
 '_songTimeOffset': 0,
 '_shuffle': 0,
 '_shufflePeriod': 0.5,
 '_previewStartTime': 12,
 '_previewDuration': 10,
 '_songFilename': 'song.egg',
 '_coverImageFilename': 'cover.jpg',
 '_environmentName': 'NiceEnvironment',
 '_customData': {'_contributors': [],
  '_customEnvironment': '',
  '_customEnvironmentHash': ''},
 '_difficultyBeatmapSets': [{'_beatmapCharacteristicName': 'Standard',
   '_difficultyBeatmaps': [{'_difficulty': 'Easy',
     '_difficultyRank': 1,
     '_beatmapFilename': 'Easy.dat',
     '_noteJumpMovementSpeed': 12,
     '_noteJumpStartBeatOffset': 0,
     '_customData': {'_difficultyLabel': '',
      '_editorOffset': 0,
      '_editorOldOffset': 0,
      '_warnings': [],
      '_information': [],
      '_suggestions': [],
      '_requirements': []}},
    {'_difficulty': 'Normal',
     '_difficultyRank': 3,
     '_b